# Part 1: Generating data with `FHI-aims` (supporting notebook)

In [1]:
from os.path import exists, join
import chemiscope

import numpy as np
import metatensor as mts
 
from rholearn.utils import cube, system
from rholearn.utils.io import unpickle_dict

from dft_settings import *

In [ ]:
# Display the nuclear geometries with chemiscope
frames = system.read_frames_from_xyz(XYZ, FRAME_IDXS)
chemiscope.show(system.chemfiles_frame_to_ase_frame(frames), mode="structure")

## 1.2: Converge SCF

In [ ]:
# Run SCF
from rholearn.aims_interface import scf
from dft_settings import DFT_SETTINGS
from hpc_settings import HPC_SETTINGS

scf.run_scf(DFT_SETTINGS, HPC_SETTINGS)

In [4]:
# Check all SCF calculations have converged
scf_dir = lambda A: f"data/raw/{A}"

not_conv = []
for A in FRAME_IDXS:
    path = join(scf_dir(A), "aims.out")
    if not exists(path):
        not_conv.append(A)
    else:
        with open(path, "r") as f:
            if "Have a nice day." not in f.read():
                not_conv.append(A)

assert len(not_conv) == 0, f"SCF not converged for {not_conv}"

In [ ]:
# Process SCF
from rholearn.aims_interface import scf
from dft_settings import DFT_SETTINGS
from hpc_settings import HPC_SETTINGS

scf.process_scf(DFT_SETTINGS)

In [5]:
# Confirm again all SCF calculations have converged via the parsed aims.out files
for A in FRAME_IDXS:
    calc_info = unpickle_dict(join(scf_dir(A), "calc_info.pickle"))
    assert calc_info["scf"]["converged"]

In [ ]:
# WARNING: execute with care! 
# Display the electron density volumetric data. This relies on py3Dmol, which is often
# unreliable in jupyter notebooks. Better to use another external software, such as
# VESTA.

# Display the electron density of structure 0
A = 0
rhocube = cube.RhoCube(join(scf_dir(A), "cube_001_total_density.cube"))
rhocube.show_volumetric()

## 1.3: Perform RI decomposition

In [ ]:
# Set up RI
from rholearn.aims_interface import ri_fit
from dft_settings import DFT_SETTINGS
from hpc_settings import HPC_SETTINGS

ri_fit.set_up_ri_fit_sbatch(DFT_SETTINGS, HPC_SETTINGS)

In [ ]:
# Run RI
from rholearn.aims_interface import ri_fit
from dft_settings import DFT_SETTINGS
from hpc_settings import HPC_SETTINGS

ri_fit.run_ri_fit(DFT_SETTINGS, HPC_SETTINGS)

In [6]:
# Check all RI calculations have finished
ri_dir = lambda A: f"data/raw/{A}/edensity"

not_finished = []
for A in FRAME_IDXS:
    path = join(ri_dir(A), "aims.out")
    if not exists(path):
        not_conv.append(A)
    else:
        with open(path, "r") as f:
            if "Have a nice day." not in f.read():
                not_conv.append(A)

assert len(not_finished) == 0, f"RI not finished for {not_finished}"

In [ ]:
# Process RI
from rholearn.aims_interface import ri_fit
from dft_settings import DFT_SETTINGS
from hpc_settings import HPC_SETTINGS

ri_fit.process_ri_fit(DFT_SETTINGS, HPC_SETTINGS)

In [ ]:
# Inpsect the basis set definition for frame 0
processed_dir = lambda A: f"data/processed/{A}/edensity"  # relative dir

A = 0
unpickle_dict(join(processed_dir(A), "basis_set.pickle"))

In [10]:
# Inspect the metadata structure of the coefficient vector and overlap matrix
coeffs = mts.load(join(processed_dir(A), "ri_coeffs.npz"))
ovlp = mts.load(join(processed_dir(A), "ri_ovlp.npz"))

In [ ]:
# The coefficient (and projection) vector is a 1D array, stored in a block sparse
# format. As it represents an equivariant target property (i.e. the density decomposed
# onto a spherical basis), knowing the spherical symmetry of each block - in terms of
# `o3_lambda` and `o3_sigma` - is crucial for unserstanding the bahviour under rotation
# and what equivariance-preserving operations can be applied.
coeffs

In [ ]:
# The first axis of each block is the atomic samples. The intermediate axis is the
# spherical harmonics components, and the properties (last axis) is the radial basis
# function index.
coeffs.block(o3_lambda=1, o3_sigma=1, center_type=6)

In [ ]:
# The overlap matrix is stored with pairs of center types (i.e. chemical species) as
# sparse keys. These have consistent basis set definitions, so overlap matrices between
# atoms can be stacked.
ovlp

In [ ]:
# For each center type pair, the overlaps between basis functions of pairs of atoms of
# types center_1_type and center_2_type are stacked along the samples (first) axis.
ovlp.block(center_1_type=6, center_2_type=6)

In [ ]:
# Calculate the normalised MAE, averaged over all structures
nmaes = []
for A in FRAME_IDXS:
    df_error = unpickle_dict(
        join(processed_dir(A), "df_error.pickle")
    )
    nmae = 100 * df_error['abs_error'] / df_error['norm']
    nmaes.append(nmae)

print("NMAE (%)")
print("   Min  : ", np.min(nmaes))
print("   Mean : ", np.mean(nmaes))
print("   Max  : ", np.max(nmaes))